In [1]:
from pyper.viper_interface.viper_classes import PolhemusViper
from pyper.io.io_utils import extract_data_from_standard_frame
from threading import Event
from multiprocessing.pool import ThreadPool
import time
import pandas as pd

In [2]:
viper = PolhemusViper()
viper.connect()

In [3]:
viper.get_units()

The current position units are: cm
The current orientation units are: euler_degree


In [11]:
viper.get_single_pno(pno_mode="standard")

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,20379,2022-05-10T14:50:10.929763,-19.850933,-60.832882,2.44831,170.404785,-35.387463,-61.751244


In [3]:
viper.start_continuous()

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [49]:
viper.get_single_pno(pno_mode="accelerated")


[86, 80, 82, 67, 72, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 196, 245, 22, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 66, 8, 0, 188, 179, 126, 65, 92, 211, 130, 194, 226, 0, 104, 65, 43, 198, 122, 239, 74, 203, 185, 193, 205, 2, 251, 0, 114, 255, 5, 3, 99, 156, 0, 0]


,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,1504708,2022-05-10T12:44:12.300914,15.91888,-65.412811,14.500216,-7.761082e+28,-23.224262,2.305171e-38


In [5]:
import struct

In [21]:
def intlist_to_int(int_list):
    h = "".join(f"{i:02x}" for i in int_list)
    return struct.unpack("<h", bytes.fromhex(h))[0]

def fract_to_float(fract, factor):
    return (float(fract) / 32768) * factor


In [51]:
il = [205, 2]
it = intlist_to_int(il)


fract_to_float(it, 180.0)

3.9385986328125

In [48]:
fract_to_float(29120, 180.0)

159.9609375

In [4]:
viper.start_continuous(pno_mode="standard", frame_counting="reset_frames")

[86, 80, 82, 80, 52, 0, 0, 0, 0, 0, 0, 0, 134, 109, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 66, 4, 0, 193, 216, 158, 193, 249, 76, 115, 194, 70, 248, 29, 64, 53, 89, 42, 67, 23, 143, 13, 194, 40, 178, 118, 194, 0, 0, 0, 0, 51, 54, 0, 0]


In [9]:
viper.stop_continuous()

[86, 80, 82, 80, 52, 0, 0, 0, 0, 0, 0, 0, 59, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 66, 4, 0, 138, 16, 159, 193, 109, 66, 115, 194, 127, 168, 31, 64, 39, 72, 42, 67, 119, 135, 13, 194, 232, 134, 118, 194, 0, 0, 0, 0, 211, 141, 0, 0]
Continuous streaming stopped


In [5]:
stop_con = Event()
pool = ThreadPool(processes=1)
async_result = pool.apply_async(viper.read_continuous, [stop_con])

time.sleep(2)


stop_con.set()
df = async_result.get()
# df = Thread(target=viper.read_continuous, args=(dev, ep, stop_con), daemon=True).start()

In [7]:
df1 = pd.DataFrame.from_dict(df, orient="index")
df1

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
2022-05-09T18:30:21.048151,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,233,241,0,0
2022-05-09T18:30:21.054123,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,149,37,0,0
2022-05-09T18:30:21.063161,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,184,66,0,0
2022-05-09T18:30:21.071588,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,147,54,0,0
2022-05-09T18:30:21.080209,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,234,228,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-09T18:30:23.021670,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,0,72,0,0
2022-05-09T18:30:23.029976,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,170,221,0,0
2022-05-09T18:30:23.037756,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,8,231,0,0
2022-05-09T18:30:23.046990,86,80,82,80,116,0,0,0,0,0,...,166,66,0,0,0,0,133,254,0,0


In [11]:
df.keys()

KeyError: 0

In [6]:
df

[('2022-05-10T14:48:45.669131',
  [86,
   80,
   82,
   67,
   24,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   19,
   0,
   0,
   0,
   3,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   103,
   246,
   0,
   0]),
 ('2022-05-10T14:48:45.670125',
  [86,
   80,
   82,
   80,
   52,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   83,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   66,
   4,
   0,
   174,
   236,
   158,
   193,
   145,
   73,
   115,
   194,
   97,
   66,
   29,
   64,
   66,
   92,
   42,
   67,
   231,
   168,
   13,
   194,
   201,
   249,
   118,
   194,
   0,
   0,
   0,
   0,
   223,
   94,
   0,
   0]),
 ('2022-05-10T14:48:45.672130',
  [86,
   80,
   82,
   80,
   52,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   85,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   66,
   4,
   0,
   140,
   218,
   158,
   193,
   93,
   76,
   115,
   194,
   137,
   50,
   29,
   64,
   82,
   95,
   42,
   

In [7]:
df1 = pd.DataFrame()
for i in range(len(df)):
    df2 = extract_data_from_standard_frame(df[i], "continuous")
    df1 = pd.concat([df1, df2], ignore_index=True)
# df1[30:100]

In [8]:
df1

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,595,2022-05-10T14:48:45.670125,-19.865566,-60.821842,2.457176,170.360382,-35.414944,-61.743931
1,1,597,2022-05-10T14:48:45.672130,-19.856712,-60.824574,2.456209,170.372345,-35.400982,-61.739449
2,1,599,2022-05-10T14:48:45.681040,-19.84902,-60.825802,2.460265,170.378052,-35.402657,-61.733242
3,1,601,2022-05-10T14:48:45.688998,-19.842333,-60.827412,2.468853,170.378357,-35.395569,-61.712189
4,1,603,2022-05-10T14:48:45.697002,-19.841692,-60.828144,2.474027,170.376114,-35.396214,-61.699959
...,...,...,...,...,...,...,...,...,...
239,1,1073,2022-05-10T14:48:47.656540,-19.861401,-60.820068,2.491421,170.310623,-35.357346,-61.613598
240,1,1075,2022-05-10T14:48:47.664839,-19.854418,-60.824074,2.489554,170.325546,-35.358871,-61.617264
241,1,1077,2022-05-10T14:48:47.672669,-19.862791,-60.820885,2.495544,170.309494,-35.369354,-61.613808
242,1,1079,2022-05-10T14:48:47.681132,-19.869978,-60.819149,2.498252,170.29747,-35.379871,-61.614956


In [3]:
viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")

error: unpack requires a buffer of 4 bytes

In [6]:
viper.start_continuous(dev, ep, pno_mode="standard", framecount="reset")

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [4]:
dev.read(ep, 100, 200)

array('B', [86, 80, 82, 80, 84, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 66, 0, 0, 101, 231, 15, 65, 139, 200, 180, 64, 106, 101, 20, 64, 105, 254, 43, 67, 7, 125, 218, 193, 119, 110, 35, 195, 0, 0, 0, 0, 1, 130, 4, 0, 10, 166, 51, 66, 127, 109, 7, 194, 64, 72, 113, 62, 40, 135, 238, 66, 174, 8, 212, 192, 192, 81, 3, 67, 0, 0, 0, 0, 34, 165, 0, 0])

In [3]:
df = viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")
df

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,24154,2022-05-05T22:48:39.533252,9.010628,5.649452,2.351197,172.175293,-28.035522,-165.601578
1,2,24154,2022-05-05T22:48:39.533252,44.924061,-33.885765,0.258443,119.227028,-6.605873,131.344360
